<a href="https://colab.research.google.com/github/hooooooni/mHealthApp-review-textmining_colab/blob/main/OKT_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 가상환경 세팅

구글 코랩에서 기본적으로 제공하고 있는 패키지의 버전들은 해당 프로젝트를 import 하는 데에 있어 적합하지 않아 코랩에서 제공하는 Conda 가상환경을 구축해야 합니다.

## 1) Colab 가상환경 구축

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
!conda install openmm

!conda env update -n base -f environment.yml

앞으로 사용할 패키지를 설치하는 과정입니다.

In [ ]:
! pip install numpy
! pip install pandas
! pip install konlpy
! pip install tqdm
! pip install re
! pip install gensim
! pip install pyLDAvis

## 2) 패키지 import

In [5]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from konlpy.tag import Okt  # 변경: Mecab에서 Okt로
okt = Okt()
from tqdm import tqdm
import re
from gensim import corpora
import gensim
import pyLDAvis.gensim_models
from collections import Counter

## 3) 리뷰 데이터 불러오기

본인의 구글 드라이브에 기본 파일들을 옮겨 두었고, 마운트 해두었다는 가정 하에 밑의 코드들은 작동합니다.<br>
코랩 왼쪽에 보시면 폴더 표시가 있는데 폴더에 들어가셔서 마운트가 된 것을 꼭 확인하세요!

그 다음, data가 있는 곳에 우클릭을 하시고 '경로 복사'를 누르신 후 pd.read_excel()의 괄호 안 경로를 수정해 주시면 됩니다.

In [6]:
dataset_raw = pd.read_excel("/content/drive/MyDrive/dongguk/dataset_raw.xlsx") #본인 경로로 수정
dataset_raw.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,app,review,rating
0,다리 근육 운동 – 4주 프로그램,다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐,1
1,다리 근육 운동 – 4주 프로그램,굿,5
2,다리 근육 운동 – 4주 프로그램,최고입니디,5
3,다리 근육 운동 – 4주 프로그램,아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂,5
4,다리 근육 운동 – 4주 프로그램,ᆞ,5


* pd.read_excel(’엑셀 파일 디렉토리’) : 엑셀 파일 가져오기 함수

* head() : 상위 5행을 출력

**요약**

위의 과정에서는 dataset_raw는 시트 이름을 지정하고, head() 함수를 통해 불러온 시트가 잘 불러와졌나 head 함수를 통해 확인해 보았습니다.

## 4) 데이터 탐색

결측치 존재 여부, 데이터 타입, 데이터 개수를 확인

In [7]:
dataset_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540076 entries, 0 to 540075
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   app     540076 non-null  object
 1   review  540074 non-null  object
 2   rating  540076 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.4+ MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


* info() : 데이터의 정보를 출력

**요약**

행과 열의 크기, 컬럼명, 컬럼을 구성하는 값의 자료형 등을 출력했습니다.

보통 이 절차를 통해 결측치 존재 여부, 데이터 타입, 데이터 개수 등을 확인합니다.

# 2. 데이터 전처리


## 1) 결측치 확인

In [8]:
dataset_raw.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


app       0
review    2
rating    0
dtype: int64

* isnull() : 데이터프레임의 각 요소(element)가 결측값(null 또는 NaN)인지 여부를 나타내는 Boolean 값을 가지는 동일한 모양의 데이터프레임을 반환한다. 결측값이면 True, 그렇지 않으면 False가 된다.

* sum() : 각 열의 결측값(True의 개수)을 계산하여 열별로 합산합니다. 결과는 각 열에 대한 결측값의 개수를 나타내는 시리즈(Series)나 데이터프레임이 된다.

**요약**

불러온 데이터에 결측치가 있는지 확인해 보았습니다.

## 3) 결측치 제거

In [9]:
# axis = 0: 결측치 포함한 모든 행 제거
dataset = dataset_raw.dropna(axis = 0)
dataset.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


app       0
review    0
rating    0
dtype: int64

* dropna() : 결측값을 제거하는 메소드

* axis=0의 의미 ⇒ axis 값이 0일 경우 행 방향으로 작업을 수행하라는 의미이다. 따라서 행에 대해 결측값이 있는지 확인하고 결측값이 있는 행을 제거한다.

**요약**

위에서 결측치가 있는 것을 확인했기 때문에 결측치를 포함한 모든 행을 제거하고 올바르게 제거가 되었는지 확인했습니다.

## 4) 분석에서 제외할 앱 리뷰 삭제

미리 만들어 둔 제외할 앱 리스트를 로드합니다.

로드 후, 제대로 불러와졌는지 head 함수를 통해 확인합니다.

In [10]:
# 제외할 앱 리스트 Load
remove_app_list = pd.read_excel('/content/drive/MyDrive/dongguk/remove_app_list.xlsx') #본인 경로로 수정
remove_app_list.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,app
0,캐시슬라이드 스텝업 - 걸음에 포인트를 더하다
1,만보기 - 걸음 계산기
2,딱 1주일 다이어트 습관 : 요요없는 건강한 다이어트
3,타임캐시 – 돈버는 어플
4,돈버는어플 - 캐시런


분석에서 제외할 앱에 해당되는 리뷰는 분석에서 제외 시킵니다.

예를 들어, 캐시워크와 같은 경우에는 현재 프로젝트에서 진행하는 사용자 경험과는 거리가 먼 리뷰가 많습니다. 또한, 보상형 만보기 앱 리뷰는 대부분 포인트 적립 에러, 상품 결제 에러와 같은 보상이나 기술적 에러에 대한 리뷰가 많아 본 프로젝트의 목적과는 거리가 멀어 제거가 필요합니다.

**코드 분석**

위의 코드에서는 for문과 try문을 활용하여 'app' 열의 값이 **`remove_app`**이 아닌 행만 선택하여 새로운 **`dataset`**으로 할당합니다. 즉, 'app' 열의 값이 **`remove_app`**인 행을 제거합니다.

except의 경우, 즉 예외가 실행되면 아무 작업도 수행하지 않고 넘어갑니다. (제거할 앱이 아니기 때문)

In [11]:
for remove_app in remove_app_list['app']:
    try:
        dataset = dataset[dataset['app'] != remove_app]
    except:
        pass

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


현재 데이터에는 인덱스가 포함되어 있어 인덱스를 초기화합니다.

In [12]:
dataset.reset_index(drop = True, inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


선별된 리뷰를 확인합니다. 아래의 약 26만 개의 리뷰가 1차적으로 선별되었습니다.

In [13]:
dataset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,app,review,rating
0,다리 근육 운동 – 4주 프로그램,다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐,1
1,다리 근육 운동 – 4주 프로그램,굿,5
2,다리 근육 운동 – 4주 프로그램,최고입니디,5
3,다리 근육 운동 – 4주 프로그램,아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂,5
4,다리 근육 운동 – 4주 프로그램,ᆞ,5
...,...,...,...
264064,"코인스텝 - 돈버는 만보기, 건강과 캐시를 동시에 챙기자~",설치하고 로그인 하니 HTTP 500 Internal server error 가 뜨...,2
264065,"코인스텝 - 돈버는 만보기, 건강과 캐시를 동시에 챙기자~",위젯도 만들어주면 좋은데...... 제가 다른 걷기 위젯으로 사용 하는 앱이 있는데...,3
264066,"코인스텝 - 돈버는 만보기, 건강과 캐시를 동시에 챙기자~",이거 erc223 기반이라는데 토큰이 현매 이더리움위에 올라가져 있나요?,5
264067,"코인스텝 - 돈버는 만보기, 건강과 캐시를 동시에 챙기자~",좋은 만보기네요,5


**요약**

해당 프로젝트의 목적과 맞지 않는 앱은 삭제하고자 하는데, 미리 로드해 둔 remove_app_list(분석 제외 앱 리스트)를 활용해 ‘app’ 열의 값이 remove_app인 행을 제거했습니다.

## 5) 전처리용 Dictionary 불러오기

불용어 리스트를 불러옵니다. (방법은 지금까지 한 방법과 동일합니다.)

In [14]:
stopword_list = pd.read_excel('/content/drive/MyDrive/dongguk/stopword_list.xlsx') #본인 경로로 수정
stopword_list.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,stopword
0,가까스로
1,가량
2,가령
3,가민
4,가민커넥트


데이터 치환 리스트를 불러옵니다. (방법은 지금까지 한 방법과 동일합니다.)

In [15]:
replace_list = pd.read_excel('/content/drive/MyDrive/dongguk/replace_list.xlsx') #본인 경로로 수정
replace_list.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,before_replacement,after_replacement
0,S헬스,삼성헬스
1,LG폰,스마트폰
2,LG V10,스마트폰
3,G7,스마트폰
4,GX,그룹운동


한 글자인 키워드 리스트를 불러옵니다. (방법은 지금까지 한 방법과 동일합니다.)

In [16]:
one_char_keyword = pd.read_excel('/content/drive/MyDrive/dongguk/one_char_list.xlsx') #본인 경로로 수정
one_char_keyword.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,one_char_keyword
0,컵
1,방
2,물
3,돈
4,꿈


**요약**

전처리를 위해 미리 만들어 둔 불용어 리스트, 데이터 치환 리스트, 한 글자 키워드 리스트를 불러왔습니다.

## 6) 단어 치환

같은 의미의 단어를 하나의 단어로 통일하는 작업입니다. LDA 토픽 모델링은 빈출 어휘를 중심으로 결과를 제공하기 때문에 단어를 통일할 필요가 있습니다. 아래와 같이 단어 치환을 위한 함수(replace_word)를 작성하였습니다.

In [17]:
def replace_word(review):
    for i in range(len(replace_list['before_replacement'])):
        try:
            # 치환할 단어가 있는 경우에만 데이터 치환 수행
            if replace_list['before_replacement'][i] in review:
                review = review.replace(replace_list['before_replacement'][i], replace_list['after_replacement'][i])
        except Exception as e:
            print(f"Error 발생 / 에러명: {e}")
    return review

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**코드 분석**

1. `for i in range(len(replace_list['before_replacement'])):` : before_replacement의 길이만큼 for문을 실행합니다.
2. `try ~ except 전까지` : 치환할 단어가 있는 경우에만 텍스트에서 해당 단어를 치환합니다. **`replace_list`**의 'after_replacement' 열에 있는 값으로 해당 단어를 치환합니다.
3. `except Exception as e:` : 예외가 발생하면 해당 예외에 대한 정보를 출력합니다.
4. `return review` : 치환 작업이 완료된 텍스트를 반환합니다.

치환할 단어를 포함한 문장을 찾아 해당되는 모든 단어를 치환하고, 데이터셋 데이터프레임 내 새로운 칼럼(review_prep)에 저장합니다 (이 과정은 시간이 조금 소요가 됩니다!)

In [18]:
dataset['review_prep'] = ''
review_replaced_list = []
for review in tqdm(dataset['review']):
    review_replaced = replace_word(str(review)) # 문자열 데이터 변환
    review_replaced_list.append(review_replaced)
dataset['review_prep'] = review_replaced_list
dataset.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 264069/264069 [13:36<00:00, 323.38it/s]


,app,review,rating,review_prep
0,다리 근육 운동 – 4주 프로그램,다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐,1,다른 P4P어플과 연동 하면 기존에 있던 스케줄이 싹 사라짐
1,다리 근육 운동 – 4주 프로그램,굿,5,굿
2,다리 근육 운동 – 4주 프로그램,최고입니디,5,최고입니디
3,다리 근육 운동 – 4주 프로그램,아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂,5,아무곳에서나 확인 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂
4,다리 근육 운동 – 4주 프로그램,ᆞ,5,ᆞ


**코드 분석**

1. `dataset['review_prep'] = ''` : 'review_prep'이라는 빈 열을 데이터프레임에 추가합니다.
2. `review_replaced_list = []` : 빈 리스트 `review_replaced_list`를 생성합니다. 이 리스트는 치환된 리뷰들을 저장하기 위해 사용됩니다.
3. `for review in tqdm(dataset['review']):`: 데이터프레임의 'review' 열에 대해 반복문을 실행합니다. 각 리뷰에 대해 반복합니다.
4. `review_replaced = replace_word(str(review))`: 각 리뷰에 대해 `replace_word` 함수를 호출하여 특정 단어들을 치환한 후, 그 결과를 `review_replaced`에 저장합니다. `replace_word` 함수는 앞서 정의된 단어 치환 로직을 수행합니다.
5. `review_replaced_list.append(review_replaced)`: `review_replaced`를 `review_replaced_list`에 추가합니다. 이 리스트에는 치환된 리뷰들이 저장됩니다.
6. `dataset['review_prep'] = review_replaced_list`: 'review_prep' 열에 `review_replaced_list`를 대입하여, 치환된 리뷰들로 열을 채웁니다.
7. `dataset.head()`: 데이터프레임의 처음 5개 행을 출력하여 확인합니다.

**요약**

같은 의미의 단어를 하나의 단어로 통일하는 작업을 진행했습니다.

LDA 토픽 모델링은 빈출 어휘를 중심으로 결과를 제공하기 때문에 단어를 통일할 필요가 있어 치환할 단어를 포함한 문장을 찾아 해당되는 모든 단어를 치환하고 review_prep라는 새로운 칼럼에 저장했습니다.

## 7) 한글 외 텍스트 제거

In [ ]:
숫자, 특수문자, 영문 등 서비스 경험과 관련된 의미를 추출해 내기 어려운 모든 문자열을 제거합니다.

In [19]:
review_removed = list(map(lambda review: re.sub('[^가-힣 ]', '', review), dataset['review_prep']))
dataset['review_prep'] = review_removed

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



* re.sub() : 정규 표현식에 매치되는 부분을 공백으로 치환하여 해당 리뷰의 문자열을 반환

* lambda : 함수를 사용하여 각 리뷰에 대해 정규 표현식을 적용한다. 위의 코드에서는 정규 표현식 **`'[^가-힣 ]'`**은 한글 및 공백을 제외한 모든 문자를 나타낸다.

* map() : 주어진 시퀀스(리스트, 튜플 등)의 각 요소에 대해 지정된 함수를 적용하여 결과를 반환하는 역할. 주로 리스트의 각 요소에 대해 동일한 작업을 적용하고자 할 때 사용합니다.

* list() : 리스트로 변환하여 저장. 위의 코드에서는 map()의 결과를 리스트로 변환하여 리스트에 저장

**요약**

한글 외 특수문자, 숫자, 영문 등은 서비스 경험과 관련된 의미를 추출하기 어려워 lambda를 활용해 각 리뷰에 대해 정규 표현식을 적용하여 한글 외의 텍스트를 제거했습니다.

## 8) 평점 기준 데이터 분리

구글 플레이 스토어는 평점이 5점 만점입니다. 따라서 본 프로젝트에서는 4~5점을 긍정적인 평가를 받은 리뷰로, 1~2점을 부정적인 평가를 받은 리뷰로 분류하였습니다. 이는 서비스 이용 경험과 관련한 리뷰 내용이 긍정적인지, 부정적인지를 구분하기 위한 목적입니다.

In [20]:
# 긍정적 리뷰(평점 5점 만점 기준 4, 5점)
review_pos = dataset[(4 == dataset['rating']) | (dataset['rating'] == 5)]['review_prep']
# 부정적 리뷰(평점 5점 만점 기준 1, 2점)
review_neg = dataset[(1 == dataset['rating']) | (dataset['rating'] == 2)]['review_prep']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**요약**

review_pos에는 4,5점을 받은 리뷰로 분류하고,

review_neg에는 1,2점을 받은 리뷰로 데이터를 분류했습니다.

## 9) 토큰화

원본 프로젝트에서는 KoNLPy에서 속도 및 토큰화 측면에서 성능이 가장 우수한 Mecab 형태소 분석기를 활용했습니다. 하지만 코랩 환경에서는 동작하기 어렵기 때문에 okt 형태소 분석기를 활용했습니다.<br>
특히, 명사가 문장 내 맥락을 파악하는 데 핵심 형태소이며 빈출 어휘를 쉽게 파악할 수 있다는 장점이 있기 때문에, 리뷰에서 명사만 추출하였습니다.

이것도 시간이 좀 걸리는 작업입니다!

In [21]:
review_tokenized_pos = list(map(lambda review: okt.nouns(review), review_pos))
review_tokenized_neg = list(map(lambda review: okt.nouns(review), review_neg))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**코드 분석**

1. `review_tokenized_pos = list(map(lambda review: mecab.nouns(review), review_pos))`:
    - `review_pos`라는 리스트(또는 시퀀스)에 있는 각 리뷰에 대해 `mecab.nouns()` 함수를 적용합니다.
    - `lambda` 함수를 사용하여 각 리뷰에 대해 `mecab.nouns()` 함수를 호출합니다. `mecab.nouns()` 함수는 해당 리뷰에서 명사만 추출합니다.
    - `map()` 함수를 사용하여 각 리뷰에 대해 `mecab.nouns()` 함수를 적용하고, 그 결과를 새로운 리스트인 `review_tokenized_pos`에 저장합니다.
2. `review_tokenized_neg = list(map(lambda review: mecab.nouns(review), review_neg))`:
    - `review_neg`라는 리스트(또는 시퀀스)에 있는 각 리뷰에 대해 `mecab.nouns()` 함수를 적용합니다. 마찬가지로 `lambda` 함수를 사용하여 각 리뷰에 대해 `mecab.nouns()` 함수를 호출하여 명사만 추출합니다.
    - `map()` 함수를 사용하여 각 리뷰에 대해 `mecab.nouns()` 함수를 적용하고, 그 결과를 새로운 리스트인 `review_tokenized_neg`에 저장합니다.

**요약**

긍정적인 리뷰와 부정적인 리뷰에서 명사만 추출하여 각각 review_tokenized_pos와 review_tokenized_neg에 저장했습니다.

## 10) 불용어 제거

토큰화 후 토큰이 2글자인 경우 불용어에 해당되지 않는 토큰만 분석에 활용합니다. 토큰이 1글자인 경우에는 1글자 키워드 리스트에 포함되는 토큰만 분석에 활용합니다.

In [22]:
def remove_stopword(tokens):
    review_removed_stopword = []
    for token in tokens:
        # 토큰의 글자 수가 2글자 이상인 경우
        if 1 < len(token):
            # 토큰이 불용어가 아닌 경우만 분석용 리뷰 데이터로 포함
            if token not in list(stopword_list['stopword']):
                review_removed_stopword.append(token)
        # 토큰의 글자 수가 1글자인 경우
        else:
            # 1글자 키워드에 포함되는 경우만 분석용 리뷰 데이터로 포함
            if token in list(one_char_keyword['one_char_keyword']):
                review_removed_stopword.append(token)
    return review_removed_stopword

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**코드 분석**

1. `def remove_stopword(tokens):`: `remove_stopword`라는 함수를 정의합니다. 이 함수는 `tokens`라는 매개변수를 받습니다. `tokens`는 불용어를 제거할 대상입니다.
2. `review_removed_stopword = []`: 빈 리스트 `review_removed_stopword`를 생성합니다. 이 리스트에는 불용어가 제거된 토큰들이 저장됩니다.
3. `for token in tokens:`: `tokens` 리스트에 대해 반복문을 실행합니다. 각 토큰에 대해 반복합니다.
4. `if 1 < len(token):`: 토큰의 길이가 1보다 큰지 확인합니다. 즉, 토큰이 2글자 이상인 경우에 해당하는 조건입니다.
5. `if token not in list(stopword_list['stopword']):`: 토큰이 불용어 리스트에 없는지 확인합니다. `stopword_list`에 있는 불용어는 제외하고, 불용어가 아닌 토큰만 `review_removed_stopword`에 추가됩니다. (아닌 토큰만 분석용 리뷰 데이터로 포함하기 위함)
6. `else: if token in list(one_char_keyword['one_char_keyword']):`: 토큰의 길이가 1인 경우, 즉 1글자인 경우, 토큰이 1글자 키워드에 포함되는지 확인합니다. 1글자 키워드에 속하면 `review_removed_stopword`에 추가됩니다.
7. `return review_removed_stopword`: 불용어가 제거된 토큰들이 저장된 `review_removed_stopword` 리스트를 반환합니다.

In [23]:
review_removed_stopword_pos = list(map(lambda tokens : remove_stopword(tokens), review_tokenized_pos))
review_removed_stopword_neg = list(map(lambda tokens : remove_stopword(tokens), review_tokenized_neg))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


`review_tokenized_pos` 리스트에 있는 각 토큰 리스트에 대해 `remove_stopword` 함수를 적용합니다.

`lambda` 함수를 사용하여 각 토큰 리스트에 대해 `remove_stopword` 함수를 호출합니다. 이렇게 하면 각 토큰 리스트에서 불용어가 제거된 토큰들의 리스트가 반환됩니다.

`review_tokenized_neg` 리스트에 있는 각 토큰 리스트에 대해도 마찬가지로 `remove_stopword` 함수를 적용하여 불용어를 제거합니다.

**요약**

토큰화 후 토큰이 2글자인 경우 불용어에 해당되지 않는 토큰만 분석에 활용합니다. 토큰이 1글자인 경우에는 1글자 키워드 리스트에 포함되는 토큰만 분석에 활용합니다. (아닌 경우 모두 제거했음)

이후, 긍정적인 리뷰review_tokenized_pos와 부정적인 리뷰 review_tokenized_neg에서 불용어를 제거하여 각각 review_removed_stopword_pos와 review_removed_stopword_neg에 저장했습니다.

## 11) 특정 토큰 개수의 리뷰 선별

In [24]:
MIN_TOKEN_NUMBER = 3 # 최소 토큰 개수
MAX_TOKEN_NUMBER = 15 # 최대 토큰 개수

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


일반적으로, 리뷰의 길이가 길수록 사용자 경험이나 기술적 문제 등 사용자 의견이 많이 내포되어 있을 가능성이 높습니다. 하지만, 오히려 지나치게 길이가 긴 리뷰는 주제 파악이나 리뷰 내 단어 간의 조합을 활용하여 특징을 추출하는 데 어려움이 있을 수 있습니다[(Vasa et al., 2012)](https://dl.acm.org/doi/abs/10.1145/2414536.2414577?casa_token=OQqZbuik4RwAAAAA:_FOT4Fl0bj1reHDV7BNObsGIM1XR6Z7o3TZyBQeIoIX6aEGcUMKhUSMOQv0ytkc9lu3_x7ZI6jDbTVc).

따라서 본 프로젝트에서는 각 리뷰에서 추출된 명사의 개수가 `3개 이상 15개 이하`인 리뷰만을 분석에 활용하였습니다.

In [25]:
def select_review(review_removed_stopword):
    review_prep = []
    for tokens in review_removed_stopword:
        if MIN_TOKEN_NUMBER <= len(tokens) <= MAX_TOKEN_NUMBER:
            review_prep.append(tokens)
    return review_prep

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**코드 분석**

1. `def select_review(review_removed_stopword):`: `select_review`라는 함수를 정의합니다. 이 함수는 `review_removed_stopword`라는 매개변수를 받습니다. `review_removed_stopword`는 불용어가 제거된 토큰들로 이루어진 리스트입니다.
2. `review_prep = []`: 빈 리스트 `review_prep`를 생성합니다. 이 리스트에는 선택된 리뷰가 저장됩니다.
3. `for tokens in review_removed_stopword:`: `review_removed_stopword` 리스트에 있는 각 토큰 리스트에 대해 반복문을 실행합니다. 각 토큰 리스트는 불용어가 제거된 토큰들을 포함합니다.
4. `if MIN_TOKEN_NUMBER <= len(tokens) <= MAX_TOKEN_NUMBER:`: 토큰 리스트의 길이가 `MIN_TOKEN_NUMBER`보다 크고 `MAX_TOKEN_NUMBER`보다 작거나 같은지 확인합니다. 이는 토큰의 길이가 특정 범위에 있는지 검사하는 조건입니다.
5. `review_prep.append(tokens)`: 만약 토큰의 길이가 조건에 맞으면 해당 토큰 리스트를 `review_prep`에 추가합니다.
6. `return review_prep`: 선택된 리뷰들이 저장된 `review_prep` 리스트를 반환합니다.

In [26]:
review_prep_pos = select_review(review_removed_stopword_pos)
review_prep_neg = select_review(review_removed_stopword_neg)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


* select_review() : 함수에서 정의된 조건에 따라 길이가 일정 범위 내에 있는 토큰들이 선택

**요약**

리뷰의 길이가 길수록 사용자 경험이나 기술적 문제 등 사용자 의견이 많이 내포되어 있을 가능성이 높습니다. 하지만, 오히려 지나치게 길이가 긴 리뷰는 주제 파악이나 리뷰 내 단어 간의 조합을 활용하여 특징을 추출하는 데 어려움이 있을 수 있습니다.

그래서 각 리뷰에서 추출된 명사가 3개 이상 15개 이하인 리뷰만 추출하여 review_prep에 추가했습니다.

이후, review_prep에서 긍정적 리뷰는 review_prep_pos에 추가하고, 부정적 리뷰는 review_prep_neg에 추가했습니다.

## 12) 전처리 결과 확인

In [27]:
review_num_pos = len(review_prep_pos)
review_num_neg = len(review_prep_neg)
review_num_tot = review_num_pos + review_num_neg

print(f"분석한 리뷰 총 개수: {review_num_tot}")
print(f"긍정적 리뷰: {review_num_pos}개({(review_num_pos/review_num_tot)*100:.2f}%)")
print(f"부정적 리뷰: {review_num_neg}개({(review_num_neg/review_num_tot)*100:.2f}%)")

분석한 리뷰 총 개수: 84012
긍정적 리뷰: 72046개(85.76%)
부정적 리뷰: 11966개(14.24%)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


분석한 리뷰의 개수를 len함수를 통해 알아보겠습니다.

review_num_pos와 review_num_neg를 더하면 총 리뷰가 되기 때문에 review_num_tot를 생성하고 print 함수를 통해 총 리뷰 개수와 긍정적 리뷰, 부정적 리뷰의 개수를 알아 보겠습니다.

그 결과, 최초에 수집한 54만 개의 리뷰 데이터 중 전처리 후 남은 데이터는 8.4만 개입니다.

**요약**

len함수와 print 함수를 통해 분석한 리뷰의 총 개수는 84012개이고 긍정적 리뷰의 개수와 비율, 부정적 리뷰의 개수와 비율을 알 수 있었습니다.

# 3. LDA 토픽 모델링

## 1) 하이퍼파라미터 튜닝

토픽의 개수와 학습 횟수는 하이퍼파라미터입니다. 본 프로젝트에서는 긍정적 리뷰와 부정적 리뷰 각각에서 토픽 10개씩을 추출합니다. 학습 횟수는 PASSES 옵션으로 변경하며 전체 문서 데이터를 몇 번 학습할지 결정합니다.

* 하이퍼파라미터 : 모델을 학습시키기 위해 사전에 사용자가 설정해야 하는 매개변수들

* PASSES : 모델을 토픽 모델에 반복하여 입력하고 학습하는 단계의 수

In [28]:
NUM_TOPICS = 10 # 토픽 개수는 하이퍼파라미터
# passes: 딥러닝에서 Epoch와 같은 개념으로, 전체 corpus로 모델 학습 횟수 결정
PASSES = 15

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 2) 모델 학습 함수

lda_modeling은 우리가 만들 토픽 모델링을 수행하는 함수입니다.

In [29]:
def lda_modeling(review_prep):
    # 단어 인코딩 및 빈도수 계산
    dictionary = corpora.Dictionary(review_prep)
    corpus = [dictionary.doc2bow(review) for review in review_prep]
    # LDA 모델 학습
    model = gensim.models.ldamodel.LdaModel(corpus,
                                            num_topics = NUM_TOPICS,
                                            id2word = dictionary,
                                            passes = PASSES)
    return model, corpus, dictionary

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**코드 분석**

1. `def lda_modeling(review_prep):`: `lda_modeling`이라는 함수를 정의하며, `review_prep`라는 매개변수를 받습니다. 이 매개변수는 토픽 모델링을 수행할 대상인 전처리된 리뷰 데이터입니다.
2. `dictionary = corpora.Dictionary(review_prep)`: `corpora.Dictionary`를 사용하여 단어 사전(dictionary)을 생성합니다. 이 사전은 전처리된 리뷰 데이터에 있는 단어들을 고유한 정수로 매핑합니다.
    
    * corpora.Dictionary : Gensim 라이브러리에서 제공하는 도구로, 텍스트 문서를 표현하는 딕셔너리를 생성하는 역할
    
    - corpora 사용 예시
        
        ```python
        from gensim import corpora
        
        # 간단한 텍스트 문서들
        documents = [
            ['apple', 'banana', 'cherry'],
            ['banana', 'orange', 'kiwi'],
            ['apple', 'cherry', 'orange', 'banana']
        ]
        
        # Dictionary 생성
        dictionary = corpora.Dictionary(documents)
        
        # 각 단어의 고유 ID 확인
        print(dictionary.token2id)  # 단어에 대한 ID 매핑
        
        # 특정 단어의 ID 얻기
        print(dictionary.token2id.get('banana'))  # banana에 대한 ID
        ```
        
        이렇게 생성된 딕셔너리를 사용하여 텍스트 데이터를 수치화하여 다양한 자연어 처리 작업에 활용할 수 있습니다.
        
3. `corpus = [dictionary.doc2bow(review) for review in review_prep]`: 단어 빈도수 계산을 위해 `doc2bow`를 사용하여 각 리뷰를 단어 빈도수 형태(corpus)로 변환합니다. 이 형태는 단어의 인덱스와 해당 단어의 빈도수를 포함합니다.
    
    * doc2bow : Gensim 라이브러리에서 제공하는 메서드로, 문서를 Bag-of-Words 형태로 변환하는 역할을 합니다. Bag-of-Words는 텍스트 데이터를 단어의 등장 빈도로 표현한 것으로, 단어의 등장 순서는 고려하지 않습니다.
    
    - doc2bow 사용 예시
        
        ```python
        from gensim import corpora
        
        # Dictionary 생성 (단어에 ID 부여)
        dictionary = corpora.Dictionary([
            ['apple', 'banana', 'cherry'],
            ['banana', 'orange', 'kiwi'],
            ['apple', 'cherry', 'orange', 'banana']
        ])
        
        # 문서 예시
        document = ['apple', 'banana', 'banana', 'cherry']
        
        # 문서를 Bag-of-Words로 변환
        bow_document = dictionary.doc2bow(document)
        print(bow_document)
        ```
        
        출력
        
        ```python
        [(0, 1), (1, 2), (2, 1)]
        ```
        
4. LDA 모델 학습:
    - `model = gensim.models.ldamodel.LdaModel(...)`: `gensim` 라이브러리를 사용하여 LDA 모델을 생성합니다.
    - `corpus`: 앞서 변환한 단어 빈도수 형태(corpus)를 입력 데이터로 사용합니다.
    - `num_topics = NUM_TOPICS`: 토픽의 개수를 설정합니다. `NUM_TOPICS`는 미리 정의된 상수(10 하이퍼파라미터 튜닝에서 정의함)로, 토픽의 개수를 나타냅니다.
    - `id2word = dictionary`: 앞서 생성한 단어 사전을 사용하여 단어와 단어의 인덱스를 연결합니다.
    - `passes = PASSES`: LDA 모델 학습을 위해 데이터를 반복적으로 사용하는 횟수를 나타냅니다. `PASSES`는 미리 정의된 상수(15 하이퍼파라미터 튜닝에서 정의함)로, 학습 반복 횟수를 나타냅니다.
5. `return model, corpus, dictionary`: 학습된 LDA 모델(`model`), 단어 빈도수 형태(`corpus`), 그리고 단어 사전(`dictionary`)을 반환합니다.

## 3) 토픽별 단어 구성 출력 함수

In [66]:
import pandas as pd
from google.colab import files

def print_topic_prop(topics, RATING):
    topic_values = [topic[1] for topic in topics]
    topic_prop = pd.DataFrame({"topic_num": list(range(1, NUM_TOPICS + 1)), "word_prop": topic_values})

    # Excel 파일로 저장, 본인 경로로 수정 필요
    file_path = f'/content/drive/MyDrive/dongguk/topic_prop_{RATING}.xlsx'
    topic_prop.to_excel(file_path)

    # 파일을 다운로드
    files.download(file_path)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


토픽별로 구성된 단어의 비율을 출력하는 함수입니다.

**코드 분석**

1. `def print_topic_prop(topics, RATING):`: `print_topic_prop`이라는 함수를 정의하며, `topics`와 `RATING`이라는 두 개의 매개변수를 받습니다. `topics`는 각 토픽의 단어 분포를 나타내는 정보를 담고 있는 리스트이고, `RATING`은 토픽 분석을 수행한 리뷰의 평가 등급을 나타냅니다.
2. `topic_values = []`: 각 토픽의 단어 분포 값을 저장할 빈 리스트 `topic_values`를 생성합니다.
3. 토픽별로 단어 분포 값을 `topic_values`에 저장:
    - `for topic in topics:`: `topics` 리스트에 있는 각 토픽에 대해 반복합니다.
    - `topic_value = topic[1]`: 각 토픽에서 단어 분포 값을 가져와 `topic_value`에 저장합니다.
    - `topic_values.append(topic_value)`: `topic_value`를 `topic_values` 리스트에 추가합니다.
4. 토픽별 단어 분포를 데이터프레임으로 정리하고 엑셀 파일로 출력:
    - `topic_prop = pd.DataFrame(...)`: 토픽 번호와 해당 토픽의 단어 분포 값을 데이터프레임으로 정리합니다.
    - `topic_prop.to_excel('./result/topic_prop_' + RATING + '.xlsx')`: 정리한 데이터프레임을 엑셀 파일로 저장합니다. 파일명은 `topic_prop_RATING.xlsx` 형식으로 지정됩니다. `RATING`은 평가 등급을 나타내는 변수입니다.
    - `display(topic_prop)`: 정리한 데이터프레임을 화면에 출력합니다.

## 4) 시각화 함수

In [67]:
import os
import pyLDAvis
import pyLDAvis.gensim_models

def lda_visualize(model, corpus, dictionary, RATING, result_dir):
    pyLDAvis.enable_notebook()
    result_visualized = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    pyLDAvis.display(result_visualized)

    # 시각화 결과 저장
    RESULT_FILE = os.path.join(result_dir, f'lda_result_{RATING}.html')
    pyLDAvis.save_html(result_visualized, RESULT_FILE)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 5) 긍정적 리뷰 토픽 모델링

In [68]:
model, corpus, dictionary = lda_modeling(review_prep_pos)
NUM_WORDS = 10

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [69]:
RATING = 'pos'
topics = model.print_topics(num_words = NUM_WORDS)
print_topic_prop(topics, RATING)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
lda_visualize(model, corpus, dictionary, RATING, '/content/drive/MyDrive/dongguk/')
# 마지막 요소 본인 경로로 수정 필요

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 6) 부정적 리뷰 토픽 모델링

In [72]:
model, corpus, dictionary = lda_modeling(review_prep_neg)
NUM_WORDS = 10

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [73]:
RATING = 'neg'
topics = model.print_topics(num_words = NUM_WORDS)
print_topic_prop(topics, RATING)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [75]:
lda_visualize(model, corpus, dictionary, RATING,'/content/drive/MyDrive/dongguk/')
# 마지막 요소 본인 경로로 수정 필요

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
